# Spark Setup 

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

### https://spark.apache.org/downloads.html

In [ ]:
!ls
!mkdir test
!rm -r test

sample_data


In [ ]:
!wget -q https://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!ls


sample_data  spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [ ]:
!pwd
!ls /content/

/content
sample_data  spark-3.1.2-bin-hadoop3.2	spark-3.1.2-bin-hadoop3.2.tgz


# Install Spark

In [ ]:
# Set up Spark
!pip install -q findspark
!pip install py4j

!export JAVA_HOME=$(/usr/lib/jvm/java-8-openjdk-amd64 -v 1.8)
! echo $JAVA_HOME
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init("spark-3.1.2-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

     |████████████████████████████████| 198 kB 5.0 MB/s 
/bin/bash: /usr/lib/jvm/java-8-openjdk-amd64: Is a directory



In [ ]:
!ls

sample_data  spark-3.1.2-bin-hadoop3.2	spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
spark.version

'3.1.2'

# Read data from your machine

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
import os 
os.listdir('./sample_data')

['anscombe.json',
 'README.md',
 'mnist_train_small.csv',
 'mnist_test.csv',
 'california_housing_train.csv',
 'california_housing_test.csv']

In [ ]:
dataset = spark.read.csv('./sample_data/california_housing_train.csv',inferSchema=True, header =True)

In [ ]:
dataset.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [ ]:

dataset.registerTempTable("test")

In [ ]:
dataset.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

### Spark SQL to read data

In [ ]:
output = spark.sql("select count(*) from test")

In [ ]:
output.show()

+--------+
|count(1)|
+--------+
|   17000|
+--------+



In [ ]:
output = spark.sql("select housing_median_age, count(*) from test group by housing_median_age order by housing_median_age")

In [ ]:
output.show()

+------------------+--------+
|housing_median_age|count(1)|
+------------------+--------+
|               1.0|       2|
|               2.0|      49|
|               3.0|      46|
|               4.0|     161|
|               5.0|     199|
|               6.0|     129|
|               7.0|     151|
|               8.0|     178|
|               9.0|     172|
|              10.0|     226|
|              11.0|     208|
|              12.0|     192|
|              13.0|     249|
|              14.0|     347|
|              15.0|     416|
|              16.0|     635|
|              17.0|     576|
|              18.0|     478|
|              19.0|     412|
|              20.0|     383|
+------------------+--------+
only showing top 20 rows



In [ ]:
!wget -O sf_crime.csv https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD

--2022-01-15 03:12:11--  https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD
Resolving data.sfgov.org (data.sfgov.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.sfgov.org (data.sfgov.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘sf_crime.csv’

sf_crime.csv            [        <=>         ] 525.42M  1.60MB/s    in 6m 54s  

2022-01-15 03:19:06 (1.27 MB/s) - ‘sf_crime.csv’ saved [550945238]



In [ ]:
!ls


sample_data   spark-3.1.2-bin-hadoop3.2
sf_crime.csv  spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
crims = spark.read.csv('./sf_crime.csv',inferSchema=True, header =True)

In [ ]:
crims.show()

+--------------+----------+-------------+--------------+--------------------+---------+----------+-----+----------+--------------+--------------------+-------------------+------------------+--------------------+-------------------------+----------------------------+--------------------------------+--------------------------+--------------------------------------+-----------------------------+---------------------------------+----------------------+--------------------------+------------------------------+------------------------------------------------+----------------------------------+-----------------------+---------------------------------+------------------------------------------+--------------------------------+--------------------------------------+--------------------------------------------------------+-------------------------------+----------------------+---------------+
|          PdId|IncidntNum|Incident Code|      Category|            Descript|DayOfWeek|      Date| Time|

In [ ]:
crims.registerTempTable("sf_crime")

In [ ]:
result = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")

In [ ]:
display(result)

DataFrame[category: string, Count: bigint]

In [ ]:
result.show()

+--------------------+------+
|            category| Count|
+--------------------+------+
|       LARCENY/THEFT|477975|
|      OTHER OFFENSES|301874|
|        NON-CRIMINAL|236928|
|             ASSAULT|167042|
|       VEHICLE THEFT|126228|
|       DRUG/NARCOTIC|117821|
|           VANDALISM|114718|
|            WARRANTS| 99821|
|            BURGLARY| 91067|
|      SUSPICIOUS OCC| 79087|
|             ROBBERY| 54467|
|      MISSING PERSON| 44268|
|               FRAUD| 41348|
|FORGERY/COUNTERFE...| 22995|
|     SECONDARY CODES| 22378|
|         WEAPON LAWS| 21004|
|            TRESPASS| 19194|
|        PROSTITUTION| 16501|
|     STOLEN PROPERTY| 11450|
|  DISORDERLY CONDUCT|  9932|
+--------------------+------+
only showing top 20 rows



# Spark HW2 for moivelense data analysis

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2022-02-09 21:33:30--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.67MB/s    in 0.2s    

2022-02-09 21:33:30 (4.67 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [ ]:
!ls

ml-latest	     ml-latest.zip  spark-3.1.2-bin-hadoop3.2
ml-latest-small.zip  sample_data    spark-3.1.2-bin-hadoop3.2.tgz


In [ ]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
!ls ml-latest-small/

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [ ]:
ratings = spark.read.csv('./ml-latest-small/ratings.csv',inferSchema=True, header =True)
movies= spark.read.load("./ml-latest-small/movies.csv", format='csv', header = True)
links= spark.read.load("./ml-latest-small/links.csv", format='csv', header = True)
tags= spark.read.load("./ml-latest-small/tags.csv", format='csv', header = True)

In [ ]:
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [ ]:
movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
links.show(5)

+-------+-------+------+
|movieId| imdbId|tmdbId|
+-------+-------+------+
|      1|0114709|   862|
|      2|0113497|  8844|
|      3|0113228| 15602|
|      4|0114885| 31357|
|      5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows



In [ ]:
tags.show(5)

+------+-------+---------------+----------+
|userId|movieId|            tag| timestamp|
+------+-------+---------------+----------+
|     2|  60756|          funny|1445714994|
|     2|  60756|Highly quotable|1445714996|
|     2|  60756|   will ferrell|1445714992|
|     2|  89774|   Boxing story|1445715207|
|     2|  89774|            MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows



In [ ]:
ratings.count()

100836

In [ ]:
count = spark.sql("select count(*) from ratings")

In [ ]:
count.show()

+--------+
|count(1)|
+--------+
|27753444|
+--------+



In [ ]:
tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1



## Part 1: Spark SQL and OLAP 

In [ ]:
# register this rating into a table in Spark SQL, and execute a Spark SQL like group by etc 
ratings.registerTempTable("ratings")
movies.registerTempTable("movies")
links.registerTempTable("links")
tags.registerTempTable("tags")

### Q1: The number of Users

In [ ]:
q1=spark.sql("SELECT COUNT(DISTINCT userID) AS The_Number_Of_Users FROM ratings")
q1.show()

+-------------------+
|The_Number_Of_Users|
+-------------------+
|                610|
+-------------------+



### Q2: The number of Movies

In [ ]:
q2=spark.sql("SELECT COUNT(DISTINCT movieID) Total_Movie_Number FROM movies")
q2.show()

+------------------+
|Total_Movie_Number|
+------------------+
|              9742|
+------------------+



### Q3:  How many movies are rated by users? List movies not rated before

In [ ]:
q3=spark.sql("SELECT COUNT(DISTINCT movieID) FROM ratings")
q3.show(5)

+-----------------------+
|count(DISTINCT movieID)|
+-----------------------+
|                   9724|
+-----------------------+



There are 9724 movies rated by users.

In [ ]:
notrated=spark.sql("SELECT * FROM movies WHERE movieID NOT IN (SELECT movieID FROM ratings)")
notrated.show(5)
#notrated stores the movies not rated before

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|   1076|Innocents, The (1...|Drama|Horror|Thri...|
|   2939|      Niagara (1953)|      Drama|Thriller|
|   3338|For All Mankind (...|         Documentary|
|   3456|Color of Paradise...|               Drama|
|   4194|I Know Where I'm ...|   Drama|Romance|War|
+-------+--------------------+--------------------+
only showing top 5 rows



### Q4: List Movie Genres

In [ ]:
#
q4=spark.sql("SELECT DISTINCT explode(split(genres,'[|]')) AS genres, title FROM movies")
q4.show()

+---------+--------------------+
|   genres|               title|
+---------+--------------------+
|    Drama|  Money Train (1995)|
| Thriller|Usual Suspects, T...|
|Animation|Snow White and th...|
|   Comedy| Pretty Woman (1990)|
|Adventure|Bedknobs and Broo...|
| Thriller|Touch of Evil (1958)|
|   Horror|   Body Parts (1991)|
|   Comedy|     In & Out (1997)|
|    Crime|King of New York ...|
|    Drama|Dream for an Inso...|
| Children|Apple Dumpling Ga...|
|    Drama|Love Is the Devil...|
|    Drama|American History ...|
|   Comedy|Hard Core Logo (1...|
|   Action| Superman III (1983)|
|   Horror|    Tarantula (1955)|
|  Mystery|Red Violin, The (...|
| Thriller|Tequila Sunrise (...|
|    Drama|    Hairspray (1988)|
|   Sci-Fi| Galaxy Quest (1999)|
+---------+--------------------+
only showing top 20 rows



### Q5: Movie for Each Category

In [ ]:
q5=spark.sql("SELECT genres, concat_ws('/',collect_set(title)) as list_of_movies "+\
             "FROM(Select explode(split(genres,'[|]')) AS genres, title From movies)"+\
             "Group By genres")
q5.show()

+------------------+--------------------+
|            genres|      list_of_movies|
+------------------+--------------------+
|             Crime|Stealing Rembrand...|
|           Romance|Vampire in Brookl...|
|          Thriller|Element of Crime,...|
|         Adventure|Ice Age: Collisio...|
|             Drama|Airport '77 (1977...|
|               War|General, The (192...|
|       Documentary|The Barkley Marat...|
|           Fantasy|Masters of the Un...|
|           Mystery|Before and After ...|
|           Musical|U2: Rattle and Hu...|
|         Animation|Ice Age: Collisio...|
|         Film-Noir|Rififi (Du rififi...|
|(no genres listed)|T2 3-D: Battle Ac...|
|              IMAX|Harry Potter and ...|
|            Horror|Underworld: Rise ...|
|           Western|Man Who Shot Libe...|
|            Comedy|Hysteria (2011)/H...|
|          Children|Ice Age: Collisio...|
|            Action|Stealing Rembrand...|
|            Sci-Fi|Push (2009)/SORI:...|
+------------------+--------------

# Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

In [ ]:
ratings.show(5)
movie_ratings=ratings.drop('timestamp')
movie_ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



In [ ]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))


In [ ]:
movie_ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
+------+-------+------+
only showing top 5 rows



### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [ ]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [ ]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2])

##### Create ALS model


In [ ]:
als = ALS(maxIter=5,rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
# https://spark.apache.org/docs/latest/ml-collaborative-filtering.html


##### Tune model using ParamGridBuilder

In [ ]:
paramGrid = (ParamGridBuilder()
             .addGrid(als.regParam, [0.05, 0.1, 0.3, 0.5])
             .addGrid(als.rank, [5, 10, 15])
             .addGrid(als.maxIter, [1, 5, 10])
             .build())

##### Define evaluator as RMSE

In [ ]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

##### Build Cross validation 

In [ ]:
# Build Cross validation 
cv= CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

##### Fit ALS model to training data

In [ ]:
cvmodel=cv.fit(training)

##### Extract best model from the tuning exercise using ParamGridBuilder

In [ ]:
best_model=cvmodel.bestModel

### Model testing
And finally, make a prediction and check the testing error.

In [ ]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

##### Print evaluation metrics and model parameters

In [ ]:
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank: ", str(best_model._java_obj.parent().getRank())),
print (" MaxIter: ", str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam: ", str(best_model._java_obj.parent().getRegParam()))

RMSE = 0.8806319282714045
**Best Model**
 Rank:  5
 MaxIter:  10
 RegParam:  0.1


In [ ]:
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   372|    471|   3.0| 2.9775312|
|   603|    471|   4.0| 3.2607427|
|   500|    471|   1.0| 2.9381146|
|    57|    471|   3.0| 3.5664742|
|   387|    471|   3.0|  3.165833|
|   610|    471|   4.0| 3.4954052|
|   136|    471|   4.0|  4.198786|
|   287|    471|   4.5| 2.3970962|
|   608|    471|   1.5| 2.8866148|
|   373|    471|   5.0| 3.7511425|
|    44|    833|   2.0| 1.8488071|
|   609|    833|   3.0| 1.4354801|
|   599|   1088|   2.5|  2.544386|
|   111|   1088|   3.0| 3.0597968|
|    47|   1088|   4.0| 2.9640028|
|   307|   1088|   3.0| 2.4758844|
|   509|   1088|   3.0| 3.1722302|
|   226|   1088|   1.0| 3.3051124|
|   188|   1088|   4.0|  3.931591|
|   116|   1088|   4.5| 3.5074172|
+------+-------+------+----------+
only showing top 20 rows



### Model apply and see the performance

In [ ]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6917489487318035


In [ ]:
alldata.registerTempTable("alldata")

In [ ]:
spark.sql("select * from alldata").show(10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   191|    148|   5.0| 4.9492073|
|   133|    471|   4.0| 3.2416303|
|   597|    471|   2.0|  3.778933|
|   385|    471|   4.0| 3.1743922|
|   436|    471|   3.0|   3.27417|
|   602|    471|   4.0| 3.6692405|
|    91|    471|   1.0| 2.5921195|
|   409|    471|   3.0| 3.4604487|
|   372|    471|   3.0| 2.9775312|
|   599|    471|   2.5|  2.684288|
+------+-------+------+----------+
only showing top 10 rows



In [ ]:
spark.sql("select * from movies join alldata on movies.movieId=alldata.movieId").show(10)

+-------+--------------------+------+------+-------+------+----------+
|movieId|               title|genres|userId|movieId|rating|prediction|
+-------+--------------------+------+------+-------+------+----------+
|    148|Awfully Big Adven...| Drama|   191|    148|   5.0| 4.9492073|
|    471|Hudsucker Proxy, ...|Comedy|   133|    471|   4.0| 3.2416303|
|    471|Hudsucker Proxy, ...|Comedy|   597|    471|   2.0|  3.778933|
|    471|Hudsucker Proxy, ...|Comedy|   385|    471|   4.0| 3.1743922|
|    471|Hudsucker Proxy, ...|Comedy|   436|    471|   3.0|   3.27417|
|    471|Hudsucker Proxy, ...|Comedy|   602|    471|   4.0| 3.6692405|
|    471|Hudsucker Proxy, ...|Comedy|    91|    471|   1.0| 2.5921195|
|    471|Hudsucker Proxy, ...|Comedy|   409|    471|   3.0| 3.4604487|
|    471|Hudsucker Proxy, ...|Comedy|   372|    471|   3.0| 2.9775312|
|    471|Hudsucker Proxy, ...|Comedy|   599|    471|   2.5|  2.684288|
+-------+--------------------+------+------+-------+------+----------+
only s

## Recommend moive to users with id: 575, 232. 


In [ ]:
# Generate top 10 movie recommendations for each user
userRecs = bestModel.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs =bestModel.recommendForAllItems(10)


  
userRecs.show(5)


+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[{8235, 5.1001472...|
|   463|[{3379, 5.2065144...|
|   496|[{8633, 5.093732}...|
|   148|[{67618, 4.744169...|
|   540|[{3379, 5.4950104...|
+------+--------------------+
only showing top 5 rows



In [ ]:
userRecs.registerTempTable("als_recs_temp")
final_recs = spark.sql("SELECT userId,\
                               movieIds_and_ratings.movieId AS movieId,\
                               movieIds_and_ratings.rating AS prediction\
                               FROM als_recs_temp\
                               LATERAL VIEW explode(recommendations) exploded_table AS movieIds_and_ratings")


In [ ]:
final_recs.show(5)

+------+-------+----------+
|userId|movieId|prediction|
+------+-------+----------+
|   471|   8235| 5.1001472|
|   471|   4495| 5.1001472|
|   471|   3379| 4.9312143|
|   471|   6666|  4.846133|
|   471|  26171|  4.839469|
+------+-------+----------+
only showing top 5 rows



In [ ]:
final_rec = fianl_recs.join(movie_ratings,['userId','movieId'],'left').filter(movie_ratings.rating.isNull())
display(final_rec)
#find out movies that users have not seen yet

DataFrame[userId: int, movieId: int, prediction: float, rating: float]

In [ ]:
final_rec.registerTempTable("final_rec")
movies.registerTempTable("movies_df")

In [ ]:
#for 575
user575=spark.sql("select userId, title FROM final_rec t1 LEFT JOIN movies_df t2 ON t1.movieId = t2.movieId WHERE t1.userId=575")

In [ ]:
user575.show()

+------+--------------------+
|userId|               title|
+------+--------------------+
|   575|Come and See (Idi...|
|   575|World of Tomorrow...|
|   575|    Jetée, La (1962)|
|   575|It's Such a Beaut...|
|   575|Andalusian Dog, A...|
|   575|        Frank (2014)|
|   575|  Man on Wire (2008)|
|   575|Raiders of the Lo...|
|   575|Man Bites Dog (C'...|
|   575|    Submarine (2010)|
+------+--------------------+



In [ ]:
#for 233
spark.sql("select userId, title FROM final_rec t1 LEFT JOIN movies_df t2 ON t1.movieId = t2.movieId WHERE t1.userId=233").show()

+------+--------------------+
|userId|               title|
+------+--------------------+
|   233|Discreet Charm of...|
|   233|Crossing Delancey...|
|   233| Safety Last! (1923)|
|   233| On the Beach (1959)|
|   233|Holy Mountain, Th...|
|   233|Swept Away (Travo...|
|   233|Rivers and Tides ...|
|   233|   The Artist (2011)|
|   233|Best of Youth, Th...|
|   233|Beautiful Thing (...|
+------+--------------------+



## Write the report 
motivation:
For online video resource providers, it is very important to attraction more and more customers and keep them on the website. In this case, a good movie recommandation system is important and effective for this objective. Starting from this motivation, I used data from Grouplens and try to train a model for movie recommandation based on user's seen movies and ratings.
1. step1: Explortalary data analysis:
I checked the structure of the data get a general understanding about the data such as number, genre classification and movies to recommand.
2. step2: Look deeper into the movie genres.
3. step3 : used ALS model  to train this movie recommandation model. Use grid search to find the best model and used cross validation to check our final model. 

4. step4  
At last, this model can recommand ten movies when I input a user's ID and gived the out put based on the user's ratings